# Ethiopian Sign Language - Quick Training
Run all cells at once (Runtime > Run all)

In [ ]:
# ============ STEP 1: MOUNT DRIVE & CONFIG ============
from google.colab import drive
drive.mount('/content/drive')

# CHANGE THESE PATHS!
TRAIN_DIR = "/content/drive/MyDrive/eth_frames/train"
VAL_DIR = "/content/drive/MyDrive/eth_frames/val"
MODEL_SAVE_PATH = "/content/drive/MyDrive/eth_model_robust.keras"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 40

In [ ]:
# ============ STEP 2: IMPORTS & DATA LOADING ============
import os, json, numpy as np, tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

print(f"GPU: {tf.config.list_physical_devices('GPU')}")

# Strong augmentation for brightness robustness
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.3, 1.7],
    channel_shift_range=50,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2 if not os.path.exists(VAL_DIR) else 0.0
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load data
if os.path.exists(VAL_DIR):
    train_gen = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
    val_gen = val_datagen.flow_from_directory(VAL_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)
else:
    train_gen = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True, subset='training')
    val_gen = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False, subset='validation')

num_classes = train_gen.num_classes
print(f"Classes: {num_classes}, Train: {train_gen.samples}, Val: {val_gen.samples}")

# Save labels
labels = {str(v): k for k, v in train_gen.class_indices.items()}
labels_path = MODEL_SAVE_PATH.replace('.keras', '_labels.json')
with open(labels_path, 'w', encoding='utf-8') as f:
    json.dump(labels, f, ensure_ascii=False, indent=2)
print(f"Labels saved: {labels_path}")
print(f"ETH_LABELS = {[labels[str(i)] for i in range(len(labels))]}")

In [ ]:
# ============ STEP 3: BUILD & TRAIN MODEL ============
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# Phase 1: Frozen base
print("\n=== PHASE 1: Frozen base (10 epochs) ===")
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=callbacks)

# Phase 2: Fine-tuning
print("\n=== PHASE 2: Fine-tuning ===")
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, initial_epoch=10, callbacks=callbacks)

In [ ]:
# ============ STEP 4: EVALUATE & DOWNLOAD ============
best_model = tf.keras.models.load_model(MODEL_SAVE_PATH)
val_loss, val_acc = best_model.evaluate(val_gen)
print(f"\n✅ DONE! Accuracy: {val_acc*100:.2f}%")
print(f"Model: {MODEL_SAVE_PATH}")
print(f"Labels: {labels_path}")

# Download
from google.colab import files
files.download(MODEL_SAVE_PATH)
files.download(labels_path)